In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


In [7]:
# Comprobar si CUDA está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Cargar los datos
df = pd.read_csv('train.csv')

# Verificar las columnas del DataFrame
print(df.head())
print(df.columns)

# Convertir las etiquetas a valores numéricos
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Dividir el dataset en conjunto de entrenamiento y prueba
train_texts, val_texts, train_labels, val_labels = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Restablecer los índices de los conjuntos de datos
train_texts = train_texts.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

# Verificar los tamaños de los conjuntos de datos
print(f"Train texts: {len(train_texts)}, Validation texts: {len(val_texts)}")
print(f"Train labels: {len(train_labels)}, Validation labels: {len(val_labels)}")

# Cargar el modelo y el tokenizador preentrenado
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenizar los textos
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Crear un dataset personalizado en PyTorch
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Asegurarse de que las etiquetas sean LongTensor
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta de salida para los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch para entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch para evaluación
    warmup_steps=500,                # Pasos de calentamiento para el scheduler
    weight_decay=0.01,               # Peso de decaimiento
    logging_dir='./logs',            # Carpeta de salida para los logs
    evaluation_strategy="epoch",     # Estrategia de evaluación en cada época
)

# Crear el Trainer
trainer = Trainer(
    model=model,                         # El modelo a entrenar
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=train_dataset,         # Dataset de entrenamiento
    eval_dataset=val_dataset,            # Dataset de validación
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, p.predictions.argmax(-1))},  # Métrica de precisión
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
eval_result = trainer.evaluate()

# Generar predicciones y el reporte de clasificación
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)

accuracy = accuracy_score(val_labels, pred_labels)
class_report = classification_report(val_labels, pred_labels, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(class_report)

Using device: cuda
                                             message label
0  I saw this movie in NEW York city. I was waiti...   neg
1  This is a German film from 1974 that is someth...   neg
2  I attempted watching this movie twice and even...   neg
3  On his birthday a small boys tells his mother ...   neg
4  The person who wrote the review "enough with t...   pos
Index(['message', 'label'], dtype='object')
Train texts: 20000, Validation texts: 5000
Train labels: 20000, Validation labels: 5000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/3750 [03:57<?, ?it/s]
c:\Users\vilch\OneDrive\Escritorio\5TH SEMESTER CODE\MACHINE LEARNING\PROYECTO_4\venv\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 13%|█▎        | 500/3750 [00:59<06:22,  8.50it/s]

{'loss': 0.4569, 'grad_norm': 7.401895046234131, 'learning_rate': 5e-05, 'epoch': 0.4}


 21%|██        | 780/3750 [01:33<05:42,  8.66it/s]

KeyboardInterrupt: 